In [51]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers import LSTM
from PIL import Image
import random
import math
import matplotlib.pyplot as plt

In [65]:
training_data = np.load('./output/training_data.npy')
training_labels = np.load('./output/training_dists.npy')

#pad the training data matrix so it is 100x1024

#(500, 100, 1000, 1)

# zero_padding_array = np.zeros((500,100,600,1))

In [66]:
training_data.shape

(500, 100, 1000)

In [67]:
training_labels.shape

(500, 5050)

In [68]:
#training_labels_padding_zero = np.zeros(training_data.shape)

In [69]:
#training_labels_padding_zero[:training_labels.shape[0], :training_labels.shape[1], :training_labels.shape[2]] = training_labels

In [70]:
#training_labels_padding_zero.shape

In [71]:
#training_labels = training_labels_padding_zero

In [72]:
# print(training_data.shape)
# training_data = np.concatenate([training_data, zero_padding_array], axis=2)
print(training_data.shape)
print(training_labels.shape)

samples_count = training_data.shape[0]

train_size = math.floor(0.85*training_data.shape[0])

shuffle_indices = random.sample(range(0, samples_count), samples_count)

indices_train = shuffle_indices[0:train_size]
indices_test = shuffle_indices[train_size:samples_count]

X_train = training_data[indices_train,:]
Y_train = training_labels[indices_train]

X_test = training_data[indices_test,:]
Y_test = training_labels[indices_test]

print('X_train.shape : ', X_train.shape)
print('X_test.shape : ', X_test.shape)

print('Y_train.shape : ', Y_train.shape)
print('Y_test.shape : ', Y_test.shape)

np.save('./output/X_test.npy', X_test)
np.save('./output/Y_test.npy', Y_test)

(500, 100, 1000)
(500, 5050)
X_train.shape :  (425, 100, 1000)
X_test.shape :  (75, 100, 1000)
Y_train.shape :  (425, 5050)
Y_test.shape :  (75, 5050)


In [73]:
model = Sequential()

model.add(LSTM(128, input_shape=(X_train.shape[1:]), activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='sigmoid'))
model.add(Dropout(0.2))

model.add(Dense(1000, activation='relu'))

model.add(Dense(Y_train.shape[1]))

# model.add(LSTM((1), batch_input_shape=(None,100000,1), return_sequences=True, activation='sigmoid'))
# model.add(LSTM((1), batch_input_shape=(None,100000,1), return_sequences=True, activation='sigmoid'))
# model.add(LSTM((1), batch_input_shape=(None,100000,1), return_sequences=True, activation='sigmoid'))
# model.add(LSTM((1), batch_input_shape=(None,100000,1), return_sequences=True, activation='sigmoid'))

In [74]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100, 128)          578048    
_________________________________________________________________
dropout_9 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1000)              129000    
_________________________________________________________________
dense_10 (Dense)             (None, 5050)              5055050   
Total params: 5,893,682
Trainable params: 5,893,682
Non-trainable params: 0
____________________________________________

In [75]:
opt = keras.optimizers.Adam(lr=2, decay=1e-6)

In [76]:
# (4) Compile 
#model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

In [77]:
# (5) Train
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=16, epochs=400, verbose=1)


model.save('./output/CNN_model_03.h5')

Train on 425 samples, validate on 75 samples
Epoch 1/400
425/425 [==============================] - 7s 16ms/step - loss: 328084.1744 - accuracy: 0.0000e+00 - val_loss: 18.3488 - val_accuracy: 0.0000e+00
Epoch 2/400
425/425 [==============================] - 6s 14ms/step - loss: 66.6476 - accuracy: 0.0000e+00 - val_loss: 19.0392 - val_accuracy: 0.0000e+00
Epoch 3/400
425/425 [==============================] - 6s 14ms/step - loss: 256.8261 - accuracy: 0.0000e+00 - val_loss: 18.7079 - val_accuracy: 0.0000e+00
Epoch 4/400
425/425 [==============================] - 6s 14ms/step - loss: 17.4728 - accuracy: 0.0000e+00 - val_loss: 15.1906 - val_accuracy: 0.0000e+00
Epoch 5/400
336/425 [======================>.......] - ETA: 1s - loss: 13.2724 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [46]:
training_result = model.evaluate(X_train, Y_train)
testing_result = model.evaluate(X_test, Y_test)

74/74 [==============================] - 5s 71ms/step


In [47]:
print('Training Acc: ', training_result[1])
print('Testing Acc: ', testing_result[1])

Training Acc:  0.645237147808075
Testing Acc:  0.6607540249824524


In [48]:
results = model.predict(X_test)

In [ ]:
plt.plot(history.history['loss'])
plt.show()